In [ ]:
import argparse
import random
import math

class RandomizedMotifSearch:

    def __init__(self, sequences, k, iterations, pseudocount):
        self.sequences = sequences
        self.k = k
        self.iterations = iterations
        self.pseudocount = pseudocount
        self.best_motifs = []
        self.best_score = float('inf')

    def _initialize_random_motifs(self):
        motifs = []
        for seq in self.sequences:
            start = random.randint(0, len(seq) - self.k)
            motifs.append(seq[start:start+self.k])
        return motifs

    def _create_profile(self, motifs):
        profile = {'a': [self.pseudocount]*self.k, 'c': [self.pseudocount]*self.k,
                   'g': [self.pseudocount]*self.k, 't': [self.pseudocount]*self.k}
        for motif in motifs:
            for i, char in enumerate(motif):
                profile[char][i] += 1
        total_motifs = len(motifs) + 4 * self.pseudocount
        for key, value in profile.items():
            profile[key] = [v / total_motifs for v in value]
        print(profile)
        return profile

    def _most_probable_motif(self, sequence, profile):
        max_prob = -1
        best_motif = sequence[:self.k]
        for i in range(len(sequence) - self.k + 1):
            subseq = sequence[i:i+self.k]
            prob = 1
            for j, char in enumerate(subseq):
                prob *= profile[char][j]
            if prob > max_prob:
                max_prob = prob
                best_motif = subseq
        return best_motif

    def _score_motifs(self, motifs, profile):
        score = 0
        for i in range(self.k):
            col = [profile[char][i] for char in "acgt"]
            max_freq = max(col)
            score += (1 - max_freq)
        return score

    def search(self):
        best_motifs = self._initialize_random_motifs()
        best_score = self._score_motifs(best_motifs, self._create_profile(best_motifs))
        for _ in range(self.iterations):
            motifs = self._initialize_random_motifs()
            while True:
                profile = self._create_profile(motifs)
                new_motifs = [self._most_probable_motif(seq, profile) for seq in self.sequences]
                score = self._score_motifs(new_motifs, profile)
                if score < best_score:
                    best_score = score
                    best_motifs = new_motifs[:]
                else:
                    break
            if best_score < self.best_score:
                self.best_score = best_score
                self.best_motifs = best_motifs[:]
        return self.best_motifs, self.best_score

    def consensus(self, motifs):
        consensus = ''
        for i in range(self.k):
            column = [motif[i] for motif in motifs]
            consensus += max(set(column), key=column.count)
        return consensus


if __name__ == "__main__":
    parser = argparse.ArgumentParser(description="Find consensus motif from sequences using randomized motif search.")
    parser.add_argument('-k', type=int, required=True, help="Motif length")
    parser.add_argument('-i', type=int, required=True, help="Number of iterations")
    parser.add_argument('-p', type=float, default=0.1, help="Pseudocount value")

    args = parser.parse_args()

    sequences = []  # Extract sequences from fasta
    try:
        while True:
            header = input().strip()
            sequence = input().strip()
            sequences.append(sequence)
    except EOFError:
        pass

    searcher = RandomizedMotifSearch(sequences, args.k, args.i, args.p)
    best_motifs, best_score = searcher.search()
    print(f"Best motifs: {best_motifs}")
    print(f"Consensus: {searcher.consensus(best_motifs)}")
    print(f"Score: {best_score}")
